<a href="https://colab.research.google.com/github/fpesceKU/PeptDes/blob/main/SB3_swap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install stable-baselines3[extra] localcider==0.1.18 >/dev/null

In [ ]:
import os
import threading
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from sklearn.preprocessing import OneHotEncoder
from localcider.sequenceParameters import SequenceParameters
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

In [ ]:
class PeptDes_SWAP(gym.Env):
    """Custom Environment that follows gym interface."""

    def __init__(self):
        super().__init__()
        # Define action and observation space
        # They must be gym.spaces objects
        # Example when using discrete actions:
        #self.max_episode_steps = 200
        self.alphabet = np.array(list("ARNDCQEGHILKMFPSTWYV")).reshape(-1, 1)
        self.enc = OneHotEncoder(handle_unknown='ignore').fit(self.alphabet)
        self.initial_state = 'MASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF'
        self.current_seq = list('MASASSSQRGRSGSGNFGGGRGGGFGGNDNFGRGGNFSGRGGFGGSRGGGGYGGSGDGYNGFGNDGSNFGGGGSYNDFGNYNNQSSNFGPMKGGNFGGRSSGGSGGGGQYFAKPRNQGGYGGSSSSSSYGSGRRF')
        self.L = int(len(self.initial_state))
        self.target = 0.75

        self.action_space = spaces.MultiDiscrete(np.array([self.L,self.L]))
        self.observation_space = spaces.MultiBinary([self.L, 20])
        #self.observation_space = spaces.Box(low=0, high=1, shape=(self.L, 20), dtype=int)
        #self.observation_space = spaces.Text(min_length=self.L, max_length=self.L, charset='ARNDCQEGHILKMFPSTWYV', seed=13)
        #self.observation_space = spaces.MultiDiscrete(np.array([2]*20*self.L))
        self.episode_length = -1

    def logger(self, message):
        out = open(f'worker_{threading.get_ident()}.dat','a+')
        out.write(message)
        out.close()

    def fwd_onehot(self, seq):
        return self.enc.transform(np.array(seq).reshape(-1, 1)).toarray()
    def score_prot(self):
        SeqOb = SequenceParameters(''.join(self.current_seq))
        return SeqOb.get_kappa()

    def reset(self, seed=None, options=None):
        # Convert the protein sequence to one-hot encoding
        observation = self.fwd_onehot(self.current_seq)

        info = {}

        return observation, info

    def step(self, action):
        self.current_seq[action[1]], self.current_seq[action[0]] = self.current_seq[action[0]], self.current_seq[action[1]]
        # Calculate the new score
        score = self.score_prot()
        s = ''.join(self.current_seq)
        self.logger(message=f'{s} {action[0]} {action[1]} {score}\n')

      	# Determine the termination flag
        terminated = True

        reward = -abs(score - self.target)

      	# Prepare the new observation
        new_observation = self.fwd_onehot(self.current_seq)

        info = {'score':score}

        return new_observation, reward, terminated, False, info

In [ ]:
vec_env = make_vec_env(PeptDes_SWAP, n_envs=1)

model = PPO("MlpPolicy", vec_env, learning_rate=0.001, n_steps=64, gae_lambda=0.01, verbose=2)

print(model.policy)

model.learn(total_timesteps=64*50, progress_bar=True)
#model.save("ppo")